<a href="https://colab.research.google.com/github/tgiangregorio/Medical_Genomics/blob/main/ROH.statistics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>